In [ ]:
import pandas as pd
data = pd.read_csv('vehicles/vehicles_messy.csv')

In [ ]:
data.head()

In [ ]:
#handle nulls - identify
#data.isnull()

null_cols = data.isnull().sum()
#null_cols
null_cols[null_cols > 0]

In [ ]:
#handle nulls - drop columns with too many nulls
#list(null_cols[null_cols > 10000].index)
drop_cols = list(null_cols[null_cols > 10000].index)
data = data.drop(drop_cols, axis=1)

In [ ]:
data.shape

In [ ]:
#handle nulls - fillna()
null_displ = data[(data['displ'].isnull()==True)]
null_displ = null_displ[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]
null_displ.head(60)

In [ ]:
data[['displ', 'cylinders']] = data[['displ', 'cylinders']].fillna(0)

In [ ]:
#check for internal inconsistencies
test = data[(data['cylinders']==0) & (data['displ']!=0)]
test[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]


In [ ]:
data.loc[(data['cylinders']==0) & (data['displ']!=0), 'cylinders'] = 4


In [ ]:
#find low-variance columns
import numpy as np
low_variance = []

for col in list(data._get_numeric_data()):
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)

print(low_variance)


In [ ]:
data = data.drop(low_variance, axis=1)


In [ ]:
data.shape

In [ ]:
#check for outliers
data.describe().transpose()
stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

In [ ]:
stats['comb08','IQR']

In [ ]:
outliers = pd.DataFrame(columns=data.columns)
#outliers
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

outliers

In [ ]:
#check datatypes
data.dtypes

In [ ]:
data['year'] = data['year'].astype('object')
data['year'].dtype
data.dtypes

In [ ]:
# harmonize values
print(set(data['trany']))

In [ ]:
data['trany'] = data['trany'].str.replace('-', '')
print(set(data['trany']))

In [ ]:
data['trany'] = data['trany'].str.replace('Automatic', 'Auto')
data['trany'] = data['trany'].str.replace('Auto\(', 'Auto ')
data['trany'] = data['trany'].str.replace('Manual\(', 'Manual ')
data['trany'] = data['trany'].str.replace('\(', '')
data['trany'] = data['trany'].str.replace('\)', '')
print(set(data['trany']))

In [ ]:
#drop duplicates
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))
data.shape

In [ ]:
#select relevant rows, then remove duplicates
before = len(data)
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 
                  'trany', 'drive', 'VClass','fuelType','barrels08', 
                  'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

data = data[select_columns].drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))
data.shape